# 🤺 Fence demo

This notebook demonstrates the use of various classes in this package. The core intent of this package is to provide a framework for interacting with language models in a way that is modular, extensible, and easy to use. The core classes are:
- `PromptTemplate`: A template for a prompt that can be rendered with a dictionary of variables (or using keywords). It wraps around jinja2.Template, but adds some additional functionality.
- `LLM`: A wrapper around a language model. 
- `Link`: Atomic LLM interaction wrapper. Takes in a PromptTemplate and a language model. It can be invoked with a dictionary of variables, and will return a dictionary of variables.
- `TransformationLink`: A wrapper around a `function` that takes a dictionary of variables and returns a dictionary of variables. It can be invoked with a dictionary of variables, and will return a dictionary of variables.
- `Chain`: A collection of links that are invoked in the right order based on the input and output keys for each Link.
- `LinearChain`: A sequence of links that are invoked in the order they are passed in.

Through these classes, we can interact with LLMs in varying degrees of complexity:
1. Just call the model directly
2. Use a `PromptTemplate` for reusability and abstraction
3. Use a `Link` for atomic LLM interaction
4. Use a `Chain` to execute a collection of `Links`

Below, we'll go through each of these levels of complexity and provide examples.

**Note** This notebook assumes you have access to AWS Bedrock, as we use Bedrock's Claude-instant model to fuel our LLM interactions.

In [75]:
%load_ext autoreload
from fence import Link, LLM, ClaudeInstantLLM, TransformationLink, Chain, LinearChain, PromptTemplate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## ⚙️ Setting up

In [76]:
# Get our model
claude = ClaudeInstantLLM(source='demo_notebook')

### 🪨 Level 1 - Just call the damn thing

In [77]:
# Use the invoke method to call the model
claude.invoke('Why is the sky blue?')

" The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue light there is, the more blue the sky appears."

In [78]:
# Just call the damn thing
claude('Why is the sky blue?')

" The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue wavelengths predominate, giving the sky its appearance of blue."

### 🔨 Level 2 - Use a PromptTemplate

In [79]:
# Initialize a prompt template
prompt_template = PromptTemplate('Why is the sky {{color}}?', input_variables=['color'])
print(prompt_template)

PromptTemplate(source=Why is the sky {{color}}?, input_variables=['color'])


In [80]:
# Render it with a dictionary
print(prompt_template.render({'color': 'blue'}))

# Render it with keyword arguments
print(prompt_template.render(color='red'))

# Input dict takes precedence over keyword arguments
print(prompt_template.render(input_dict={'color': 'blue'}, color='red'))

Why is the sky blue?
Why is the sky red?
Why is the sky blue?


In [81]:
# You can concatenate prompt templates, input variables are merged
prompt_template_sky = PromptTemplate('Why is the sky {{color}}?', input_variables=['color'])
prompt_template_grass = PromptTemplate('Why is the grass {{color}}?', input_variables=['color'])
prompt_template_dress = PromptTemplate('I like a dress with {{pattern}}.', input_variables=['pattern'])
combined_prompt_template = prompt_template_sky + prompt_template_grass + prompt_template_dress
print(combined_prompt_template)
print(combined_prompt_template.render({'color': 'blue', 'pattern': 'polka dots'}))

PromptTemplate(source=Why is the sky {{color}}? Why is the grass {{color}}? I like a dress with {{pattern}}., input_variables=['pattern', 'color'])
Why is the sky blue? Why is the grass blue? I like a dress with polka dots.


In [82]:
# You can customize the separator
base_template = PromptTemplate('Why is the sky {{color}}?', input_variables=['color'], separator=' FUNKY TOWN ')
additional_template = PromptTemplate('Why is the grass {{color}}?', input_variables=['color'])
print((base_template + additional_template).render({'color': 'blue'}))

Why is the sky blue? FUNKY TOWN Why is the grass blue?


### 🧠 Level 3 - Use Links (lol)

What are Links? In this context, they represent atomic components of LLM interaction. That means they should be able to be strung together to form a Chain, although they can be used independently as well.

In [83]:
# The simplest link is the Link class, which just takes a prompt template and a model
link = Link(template=prompt_template, llm=claude)
print(link)

Link: <['color']> -> <state>


In [84]:
# Invoke it using the input_dict, NOT keyword arguments
link(input_dict={'color': 'blue'}) # Or, equivalently, link.run(input_dict={'color': 'blue'})

[11:36:13] INFO     🖇️ Executing  Link with input: {'color': 'blue'}                                                                                ]8;id=838681;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=437341;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\

[11:36:14] INFO     🧐 Current state:  The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in      ]8;id=304190;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=62441;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\
                    Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than               
                    the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all                    
                    throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue wavelengths predominate,                   
                    giving the sky its blue color.                                                                                                              

{'state': " The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue wavelengths predominate, giving the sky its blue color."}

In [85]:
# By default, output is stored under 'state'. You can get a copy (e.g., for inspection of intermediate results) by passing a different output key
link = Link(template=prompt_template, llm=claude, output_key='intermediate')
link.run(input_dict={'color': 'blue'})

           INFO     🖇️ Executing  Link with input: {'color': 'blue'}                                                                                ]8;id=860492;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=357944;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\

[11:36:18] INFO     🧐 Current state:  The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in      ]8;id=316993;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=726073;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\
                    Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than               
                    the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all                    
                    throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue light there is, the more blue              
                    the sky appears.                                                                                                                            

{'state': " The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue light there is, the more blue the sky appears.",
 'intermediate': " The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue light there is, the more b

In [86]:
# You can name your links for easier debugging in logs
link = Link(template=prompt_template, llm=claude, name='sky')
link.run(input_dict={'color': 'blue'})

           INFO     🖇️ Executing <sky> Link with input: {'color': 'blue'}                                                                           ]8;id=594579;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=701323;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\

[11:36:20] INFO     🧐 Current state:  The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in      ]8;id=331035;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=390387;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\
                    Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than               
                    the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all                    
                    throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue wavelengths predominate,                   
                    giving the sky its blue color.                                                                                                              

{'state': " The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue wavelengths predominate, giving the sky its blue color."}

In [87]:
# You can also build TransformationLinks, which take a function that transforms any input_dict into a specific output
def concatenate(x, y):
    return f"{x} and {y}"

concat_link = TransformationLink(
    input_keys=["X", "Y"], function=concatenate, output_key="C"
)

concat_link.run(input_dict={"X": "Hello", "Y": "World"})

{'state': 'Hello and World', 'C': 'Hello and World'}

### 🚀 Level 4 - Use Chains (lol again)

In [88]:
# You can also build Chains, which are just a sequence of links. There are two types of chains: LinearChain and Chain. 
# LinearChain is a sequence of links, while Chain is a collection of links that are invoked in the right order based on the input and output keys for each Link.

# This is a LinearChain #
#########################

# Build some links
link_opposite = Link(
    template=PromptTemplate(
        "What's the opposite of {{A}}? Reply with a few words max.", ["A"]
    ),
    name = 'opposite',
    output_key="X",
)
link_superlative = Link(
    template=PromptTemplate(
        "What's a synonym for {{B}}. Reply with one word.", ["B"],
    ),
    name='superlative',
    output_key="Y",
)
link_poem = Link(
    template=PromptTemplate(
        "Write a poem about {{state}}. Return only the poem, beginning with the title.", ["state"]
    ),
    name='poem',
    output_key="Z",
)

# Now build a LinearChain
linear_chain = LinearChain(llm=claude, links=[link_opposite, link_superlative, concat_link, link_poem])

# Run it
result = linear_chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})

# Get the output
print(result['state'])

           INFO     🖇️ Executing <opposite> Link with input: {'A': 'A police officer', 'B': 'Hopeful'}                                              ]8;id=688919;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=474751;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\

[11:36:23] INFO     🧐 Current state:  Criminal.                                                                                                   ]8;id=223405;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=7940;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\

           INFO     🔑 State keys: ['A', 'B', 'state', 'X'] (added: ['state', 'X'], removed: [])                                                   ]8;id=874008;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=718570;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#443\443]8;;\

           INFO     🖇️ Executing <superlative> Link with input: {'A': 'A police officer', 'B': 'Hopeful', 'state': ' Criminal.', 'X': ' Criminal.'} ]8;id=338714;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=55965;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\

           INFO     🧐 Current state:  Optimistic                                                                                                  ]8;id=595377;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=120156;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\

...

           INFO     🔑 State keys: ['A', 'B', 'state', 'X', 'Y', 'C'] (added: ['C'], removed: [])                                                  ]8;id=249176;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=693199;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#443\443]8;;\

           INFO     🖇️ Executing <poem> Link with input: {'A': 'A police officer', 'B': 'Hopeful', 'state': ' Criminal. and  Optimistic', 'X': '    ]8;id=104166;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=673771;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\
                    Criminal.', 'Y': ' Optimistic', 'C': ' Criminal. and  Optimistic'}                                                                          

[11:36:29] INFO     🧐 Current state:  The Criminal's Optimism                                                                                     ]8;id=386506;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=462013;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\
                                                                                                                                                                
                    Behind these bars I sit all alone each day                                                                                                  
                    My crimes now haunt me as I pray                                                                                                            
                    For a chance at redemption, to start life anew                                                                                              
                    To find purpose and meaning in all that I do                                                                                                
                    Though the road will be hard and uphill all the way                                                                                         
                    Still my heart holds an optimism I cannot betray                                                                                            
                    For the future's unknown and each soul has worth                                                                                            
                    I'll greet a new dawn and give life on earth.                                                                                               
                    My debt I've now paid, my sentence now served                                                                                               
                    By hope I'm still guided, by hope I'm preserved                                                                                             
                    This cell cannot keep what's inside me confined                                                                                             
                    An optimistic spirit, a purpose I've found                                                                                                  
                    Though a criminal's path was the road I once trod                                                                                           
                    I see light up ahead, guided by faith in God                                                                                                
                    My optimism sees past these walls made of stone                                                                                             
                    To the life that awaits me, no longer alone.                                                                                                

           INFO     🔑 State keys: ['A', 'B', 'state', 'X', 'Y', 'C', 'Z'] (added: ['Z'], removed: [])                                             ]8;id=336441;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=529732;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#443\443]8;;\

 The Criminal's Optimism

Behind these bars I sit all alone each day  
My crimes now haunt me as I pray
For a chance at redemption, to start life anew
To find purpose and meaning in all that I do  
Though the road will be hard and uphill all the way
Still my heart holds an optimism I cannot betray
For the future's unknown and each soul has worth
I'll greet a new dawn and give life on earth.
My debt I've now paid, my sentence now served
By hope I'm still guided, by hope I'm preserved  
This cell cannot keep what's inside me confined
An optimistic spirit, a purpose I've found
Though a criminal's path was the road I once trod
I see light up ahead, guided by faith in God
My optimism sees past these walls made of stone
To the life that awaits me, no longer alone.


In [89]:
# A LinearChain will take the presence of the 'state' key into account when invoking the next link.
# A Chain will not. However, it has an extra 'feature' in the form of topological sorting. As long as a graph of links can be
# extracted from the chain, and the input keys (that are not generated in the chain) are given, the chain will invoke the links in the right order.

# This is a Chain #
###################
link_a = Link(
    template=PromptTemplate(
        "Capitalize this word: {{A}}. Only respond with the capitalized version", ["A"]
    ),
    name = 'opposite',
    output_key="X",
)
link_b = Link(
    template=PromptTemplate(
        "What's a synonym for {{B}}. Reply with one word.", ["B"],
    ),
    name='superlative',
    output_key="Y",
)
link_c = Link(
    template=PromptTemplate(
        "Combine {{X}} and {{Y}} and {{C}} in a meaningful sentence.", ["X", "Y", "C"]
    ),
    name='sentence',
    output_key="Z",
)
chain = Chain(llm=claude, links=[link_c, link_a, link_b]) # Note that we can pass the links in any order

# This is the sorted graph of links
chain._topological_sort()

[Link: superlative <['B']> -> <Y>,
 Link: opposite <['A']> -> <X>,
 Link: sentence <['X', 'Y', 'C']> -> <Z>]

In [90]:
# Now we can run it
try:
    result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})
except Exception as e:
    print(e)

The following input keys are required: {'A', 'C', 'B'}. Missing: {'C'}


In [91]:
# Woops, forgot something! There's no link that generates the 'C' key. We can pass it in though.
result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful", "C": "a dog"})

           INFO     🖇️ Executing <superlative> Link with input: {'A': 'A police officer', 'B': 'Hopeful', 'C': 'a dog'}                             ]8;id=922884;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=596029;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\

[11:36:30] INFO     🧐 Current state:  Optimistic                                                                                                  ]8;id=813213;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=966959;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\

           INFO     🖇️ Executing <opposite> Link with input: {'A': 'A police officer', 'B': 'Hopeful', 'C': 'a dog', 'state': ' Optimistic', 'Y': ' ]8;id=363376;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=376510;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\
                    Optimistic'}                                                                                                                                

           INFO     🧐 Current state:  A Police officer.                                                                                           ]8;id=882845;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=781495;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\

           INFO     🖇️ Executing <sentence> Link with input: {'A': 'A police officer', 'B': 'Hopeful', 'C': 'a dog', 'state': ' A Police officer.', ]8;id=673111;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=419381;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\
                    'Y': ' Optimistic', 'X': ' A Police officer.'}                                                                                              

[11:36:31] INFO     🧐 Current state:  Here is a sentence combining those elements:                                                                ]8;id=491885;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=441524;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\
                                                                                                                                                                
                    The optimistic police officer's dog always believed in him and helped brighten his day, even after difficult calls.                         

In [92]:
# Cycles are not allowed
link_up = Link(
    template=PromptTemplate(
        "Capitalize this word: {{up}}. Only respond with the capitalized version", ["up"]
    ),
    name = 'up',
    output_key="down",
)
link_b = Link(
    template=PromptTemplate(
        "What's a synonym for {{down}}. Reply with one word.", ["down"],
    ),
    name='down',
    output_key="up",
)
chain = Chain(llm=claude, links=[link_up, link_b])
try:
    chain._topological_sort()
except Exception as e:
    print(e)

Cycle detected in the dependency graph.
